# DEPENDENCIES

-------------------------------------------------------------------------------------------------------------------

In [1]:
#!/usr/bin/env python3
# coding: utf-8

# color pallet greens https://coolors.co/34463e-a4c3b2-cce3de-eaf4f4-f6fff8
# color pallet secondary https://coolors.co/353535-3c6e71-ffffff-d9d9d9-284b63

import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import os
import datetime
from colour import Color
from colr import color

#Orca must be installed?
#conda install -c plotly plotly-orca
#Kaleido is for exporting images from plotly
#pip install -U kaleido

# Colors

----

In [2]:
rainbow_colors = ['#F94144', '#f3722c', '#F9844A', '#f8961e', '#F9C74F', '#90BE6D', '#43AA8B', '#4D908E', '#277DA1', '#577590']
print(len(rainbow_colors))

10


In [3]:
red_orange = ['#F94144' ,'#F94A40', '#F9543C', '#F95D37', '#F96733', '#F8702F', '#F87A2B', '#F88326', '#F88D22', '#F8961E']
orange_yellow = ['#F89B23', '#F8A129', '#F8A62E', '#F8AC34', '#F9B139', '#F9B73F', '#F9BC44', '#F9C24A', '#F9C74F']
yellow_green = ['#EDC652', '#E2C556','#D6C459', '#CAC35C', '#BFC260', '#B3C163', '#A7C066', '#9CBF6A', '#90BE6D',]
green_cyan = ['#88BC70', '#87BC70', '#7FBA74', '#76B777', '#6EB57A', '#65B37E','#55B082', '#54AE84', '#4CAC88']
cyan_deepcyan = ['#43AA8B', '#44A78B', '#45A48C', '#479E8C', '#499C8D', '#4A998D', '#4B968D', '#4C968D', '#4D908E']
deepcyan_blue = ['#498E90','#458C92', '#408A94', '#3C8896', '#388599', '#34839B', '#2F819D', '#2B7F9F', '#277DA1']
blue_darkblue = ['#2C7C9F', '#327B9D', '#377A9B', '#3C7999', '#427998', '#477896', '#4C7794', '#527692', '#577590']

gradient = red_orange + orange_yellow + yellow_green + green_cyan + cyan_deepcyan + deepcyan_blue + blue_darkblue
print(len(gradient))
# gradient = red_orange + orange_yellow + yellow_green + cyan_deepcyan + deepcyan_blue + blue_darkblue
# print(len(gradient))

64


In [4]:
def convert_date(df):
    import datetime
    list_cols = [i.lower() for i in df.columns.tolist()]
    test_cols = ["date", "day", "time", "occurrence"]
    verified_cols = []
    for col in list_cols:
        for test in test_cols:
            if test in col:
                verified_cols.append(col)
    for tested in verified_cols:
        search_dates = df[tested].apply(
            lambda x: 'True' if isinstance(x, datetime.date) else 'False')
        if not search_dates.any():
            p_slash()
            print('ALL DATES ARE IN DATETIME SKIPPING')
            p_slash()
            continue
        elif (df[tested].map(type) == str).all():
            date_time = []
            for j in df[tested]:
                if len(j) == 10 and '/' in j:
                    date_time.append(
                        datetime.datetime.strptime(j, "%m/%d/%Y"))
                elif len(j) == 8 and '/' in j:
                    date_time.append(
                        datetime.datetime.strptime(j, "%m/%d/%y"))
                elif len(j) == 10 and '-' in j:
                    date_time.append(
                        datetime.datetime.strptime(j, "%Y-%m-%d"))
                else:
                    print("UNKNOWN DATE FORMAT SKIPPING FORMATTING")
                    return df
            df[tested] = date_time
            df[tested] = pd.to_datetime(df[tested]).apply(lambda x: x.date())
            df = df.sort_values(
                by=[tested]).reset_index(drop=True)
            # Want to use sort_values by tested, however, because its a loop it wont choose the primary date col
            # df = df.sort_values(
            #     by=['date', 'transaction', 'amount']).reset_index(drop=True)
            # (by=['date', 'category', 'identifier', 'amount']
        else:
            print('your dates are a mess, see your programmer')
    return df

# DATA

-------------------------------------------------------------------------------------------------------------------

In [5]:
# data_df = pd.read_csv('sample_data_month_categories.csv')
data_df = pd.read_csv('../../data/sample.csv')
col_names = []
for col in data_df.columns:
    col_names.append(col)
col_data = []
for i in col_names:
    col_data.append(data_df[i])

# Removing Payment Col

----

- Payment col is a positive float, this messes up graphs later

In [7]:
data_df

,Date,Date2,Transaction,Amount
0,04/19/2021,04/16/2021,BOJANGLES 1931 ARLINGTON PA,-$20.00
1,04/19/2021,04/16/2021,"SHELL OIL 45468787999 BOSTON, MA",-$8.46
2,04/19/2021,04/16/2021,"MAPCO 1571 FLOWER, FL",-$7.67
3,04/16/2021,04/14/2021,PHYSICIANS OFFICE 888-666-2424 AL,-$49.95
4,04/15/2021,04/14/2021,FOOD LION #8457 CHINA LAKE CA,-$69.99
5,04/15/2021,04/13/2021,PAYPAL *PURCHASE 555-634-1921 CA,-$39.99
6,04/14/2021,04/13/2021,AMZN Mktp US*FB12334 Amzn.com/billWA,-$23.45
7,04/14/2021,04/13/2021,RITE AID 1574 LITTLE ROCK AK,-$5.09
8,04/14/2021,04/12/2021,"TRELLO.COM, ATLASSIAN 888-942-1573 NY",-$7.40
9,04/13/2021,04/12/2021,VZWRLSS*APOCC VISW 800-922-0204 FL,-$220.50


In [6]:
data_df = convert_date(data_df)
data_df = data_df.loc[data_df['category'] != 'payment']
data_df = data_df.loc[data_df['category'] != 'interest']
data_df

KeyError: 'date'

# Adding Month and Day Num Cols 

---

In [ ]:
# Adding a numerical day for data, and a word value for day of week
data_df['day'] = pd.DatetimeIndex(data_df['date']).day
data_df['long_day'] = list(map(lambda x: datetime.datetime.strptime(str(x), "%d").strftime("%A"), data_df['day']))

In [ ]:
# Adding a numerical value col for month and a word value for mo
data_df['month'] = pd.DatetimeIndex(data_df['date']).month
data_df['long_month'] = list(map(lambda x: datetime.datetime.strptime(str(x), "%m").strftime("%B"), data_df['month']))

In [ ]:
# Fixing order
reorder = data_df[['month', 'day']].copy(deep=True)
data_df = data_df.drop(columns=['month','day'])
data_df['month'] = reorder['month']
data_df['day'] = reorder['day']

In [ ]:
data_df.head()

# Seed Data

---

In [ ]:
# Add this in after each run of data
#making seed data
seed_data = {}
for index, row in data_df.iterrows():
    if row['category'] not in seed_data.keys():
        seed_data[row['category']] = []#[row['identifier']]
    if row['identifier'] not in seed_data[row['category']]:
        seed_data[row['category']].append(row['identifier'])

In [ ]:
#seed_data

In [ ]:
# How to pull out each category for graphing
#data_df.loc[data_df['category'] == 'groceries']

# Whole month total for spending

---

In [ ]:
month_sum = (data_df.groupby(['month', 'long_month'])['amount'].sum()).to_frame(name='month_sum').reset_index()
month_categories = (data_df.groupby(['month', 'category', 'long_month'])['amount'].sum()).to_frame(name='month_sum').reset_index()

In [ ]:
month_sum

# Spending by Category summed into Months

----

In [ ]:
month_categories = month_categories.loc[month_categories['category'] != 'payment']
month_categories = month_categories.sort_values(by=['month', 'month_sum']).reset_index(drop=True)
month_categories.head()

# STACKED BAR GRAPH

-------------------------------------------------------------------------------------------------------------------

In [ ]:
data_df.head()

In [ ]:
type(data_df['amount'][0])

# Beginning Month Splitting

---

- Starting with April

In [ ]:
test = data_df.loc[data_df['month'] == 4]
test.tail()

In [ ]:
test = test.sort_values(by=['amount', 'day', 'category']).reset_index(drop=True)

In [ ]:
len(test['category'])

In [ ]:
len(gradient[:len(test['category'])])

In [ ]:
#gradient[:len(test['category'])][::-1]

In [ ]:
# Not enough color values to match len(test['category'] == 73) not sure why i cant reverse it yellow to red

In [ ]:
pos_amount = [float(i)*-1 for i in test['amount']]
gradient[:len(test['category'])]
import plotly.express as px
#wide_df = px.data.medals_wide()
fig = px.bar(test,
             x="day", y= pos_amount,
             color = 'category',
             #color_continuous_scale = gradient[:len(test['category'])],#[::-1],
             color_discrete_sequence = gradient[:len(test['category'])],
             labels={
                     "day": "Day of Month",
                     "y": "Amount",
                 },
             title="Total Spending by Category by Day in April")
fig.update_traces(marker_line_color='grey',
                  marker_line_width=0.5, opacity=1)
fig.update_layout(
        bargap = 0.1)
fig.show()

In [ ]:
len(gradient)

In [ ]:
# num_colors = len(cat_order['category'])
# num_colors

In [ ]:
rainbow_colors = ['#F94144', '#f3722c', '#F9844A', '#f8961e', '#F9C74F', '#90BE6D', '#43AA8B', '#4D908E', '#277DA1', '#577590']
# color= rainbow_colors[:len(cat_order['category'])+1]
# num_colors = len(cat_order['category'])

In [ ]:
# len(cat_order['category'])
# sym = ['x', '+', '.']
# patterns = (len(cat_order['category'])-((round(len(cat_order['category'])/len(sym))) * len(sym))) + (round(len(cat_order['category'])/len(sym))) * len(sym))

In [ ]:
cat_order = month_categories.sort_values(by=['month','category']).reset_index(drop=True)
pos_amount = [i*-1 for i in cat_order['month_sum']]
import plotly.express as px
fig = px.bar(month_categories,
             x= cat_order['long_month'], y= pos_amount,
             color= cat_order['category'],
             color_discrete_sequence = gradient[:len(cat_order['category'])][::-1],
             labels={
                     "x": "Month",
                     "color": 'Category',
                     "y": "Amount",
                 },
             title="Total Spending by Category by Month")
fig.update_layout(
        bargap = 0.15)
fig.show()

In [ ]:
type(data_df['amount'][0])

In [ ]:
#categories = data_df['category'].unique()
# sum_amount = data_df.loc[data_df['category'] == 'food']
# sum_amount['amount'].sum().round(2)

In [ ]:
# Need to make a DF where all the unique categories are summed

In [ ]:
# Make the date sum by month

In [ ]:
sum_categories = data_df.groupby(['category', 'date']).sum().sort_values("date")
sum_categories = sum_categories.rename(columns={'category':'categories'})
sum_categories

In [ ]:
#type(sum_categories['date'][0])

In [ ]:
#sum_categories['categories'] = sum_categories.index
sum_categories.reset_index(inplace = True)
sum_categories['amount'] = [i*-1 for i in sum_categories['amount']]
sum_categories

In [ ]:
import plotly.express as px
#wide_df = px.data.medals_wide()
fig = px.bar(sum_categories,
             x="date", y= sum_categories['category'],
             color = 'amount',
             labels = {'category':'amount'},
             title="Total Spending by Category by Day")
fig.show()

In [ ]:
import plotly.express as px
#wide_df = px.data.medals_wide()
fig = px.bar(sum_categories, x="date", y= sum_categories['amount'], color ='amount', title="Total Spending by Category by Day")
fig.show()

# ORIGINAL STACKED BAR GRAPH

-------------------------------------------------------------------------------------------------------------------

In [ ]:
import plotly.express as px
wide_df = px.data.medals_wide()
fig = px.bar(wide_df, x="nation", y=["gold", "silver", "bronze"], title="Wide-Form Input")
fig.show()

In [ ]:
print(px.data.medals_wide())

In [ ]:
month_categories.head()

In [ ]:
# import plotly.express as px
# fig = px.bar(month_categories, x="month", y='month_sum', title="Wide-Form Input", color = 'category')
# fig.show()

# TABLES

-------------------------------------------------------------------------------------------------------------------

In [ ]:
def get_length(df):
    col = ''
    for i in df.columns:
        get_length = df[i].map(len).max()
    return get_length

In [ ]:
def convert_str(series):
    return [str(i) for i in series]

In [ ]:
#Somehow because this is a jupyter notebook this function reassigns the column data that is not a string,
# to string and keeps that value within the imported dataframe
def get_max(df):
    max_num = 0
    max_col = ''
    max_list = []
    for i in range(len(df.columns)):
        if df[df.columns[i]][0] == type('s'):
            continue
        else:
            df[df.columns[i]] = convert_str(df[df.columns[i]])
        get_length = df[df.columns[i]].map(len).max()
        max_list.append(get_length)
        if max_num < get_length:
            max_num = get_length
            max_col = df.columns[i]
            col_num = i
#         if max_num < len(df.columns[i]):
#             max_num = len(df.columns[i])
    return max_num, max_col, max_list

In [ ]:
def make_table(df, colors, rowOddColor, rowEvenColor):
    if not os.path.exists('images'):
        os.mkdir('images')
    fig = go.Figure(data=[go.Table(
        columnorder = [i+1 for i in range(len(df.columns))],
        columnwidth = [i*30 for i in get_max(df)[2]],
        header=dict(values=list(df.columns[:8]),
                    fill_color=colors[0],
                    align='left',
                    font = dict(family="Arial", size=14, color= colors[1])),
        cells=dict(values=[df.date_p, df.date, df.transaction, df.account, df.amount, df.balance, df.identifier, df.category],
                   #fill_color= [[rowOddColor, rowEvenColor, rowOddColor, rowEvenColor]*4],#colors[2],
                   align='left',
                   font = dict(family="Arial", size=12, color=colors[3])))])

    fig.write_image("images/table.png", format="png")#, width=600, height=350, scale=2)
    #pdf is higher quality however it omits all the lines below
    max_height = len(df) * 50
    pio.write_image(fig, 'images/table.pdf', width=700, height=max_height)
    fig.show()

In [ ]:
type(data_df['amount'][0])

In [ ]:
len(data_df)

In [ ]:
data_df.head()

In [ ]:
# colors ['fill_color_header/bkgd', 'font_color_header/light', 'cells_color/light', 'cells_font_color/dk']
rowEvenColor = '#cce3de'
rowOddColor = '#EAF4F4'
green_colors = ['#6b9080', '#f6fff8', '#cce3de', '#34463E']
make_table(data_df.head(50), green_colors, rowEvenColor, rowOddColor)

In [ ]:
# color pallet https://coolors.co/353535-3c6e71-ffffff-d9d9d9-284b63
# colors ['fill_color_header/bkgd', 'font_color_header/light', 'cells_color/light', 'cells_font_color/dk']
blue_colors = ['#284B63','#FFFFFF', '#D9D9D9', '#353535']
white = '#FFFFFF'
grey = '#D9D9D9'
make_table(data_df.head(50), blue_colors, grey, white)

In [ ]:
type(data_df['amount'][1])

# DICTIONARY OPTIONS

-------------------------------------------------------------------------------------------------------------------

 |   |   |   |   |   | |||
|---|---|---|---|---|---|---|---|
 | **parent**       | **type**         | **priority**     | **fixed/variable** | **one-time/re-occuring** | **identifier**    | **amount**  | **date**    | **transaction**                   |
 | health       |life insurance| essential    | fixed          | re-occuring          | 'AAA Insurance| -50     | 01/22/21| AAA INSURANCE PAYMENT 48684   |
 | entertainment|movies        | non-essential| variable       | one-time             | 'REGAL'       | -15     | 01/12/21| CINEMA REGAL PARK WEST VA 1588|


In [ ]:
# Thinking about adding another column based on this data
# dataframe might look good like
# where parent = budget category
# | parent       | type         | priority     | fixed/variable | one-time/re-occuring | identifier    | amount  | date    | transaction                   |
# | health       |life insurance| essential    | fixed          | re-occuring          | 'AAA Insurance'| -50     | 01/22/21| AAA INSURANCE PAYMENT 48684   |
# | entertainment|movies        | non-essential| variable       | one-time             | 'REGAL'       | -15     | 01/12/21| CINEMA REGAL PARK WEST VA 1588|

fixed = {
    'debt':['car loan', 'student loan/tution'],
    'housing':['home owners insurance', 'renters insurance', 'rent', 'mortgage', 'property taxes'],
    'utilities':['cable', 'internet', 'garbage'],
    'medical':['health insurance'],
    'childcare':['daycare', 'child support'],
    'entertainment':['streaming services']
}

variable = {
    'utilities':['water', 'electricity', 'gas', 'phone']
}

essentials = {
    'savings' : ['life insurance', 'emergency fund', 'ROTH IRA', '401(k)', 'college fund', 'mad money'],
    'debt' : ['credit card', 'car loan', 'student loan', 'tuition', 'pay day loan', 'other' ],
    'domicile' : ['home', 'renters insurance', 'home insurance', 'rent', 'mortgage', 'repair', 'maitenance', 'furnishings', 'property taxes', 'yard', 'cleaning supply', 'kitchen ware','tools'],
    'transportation' : ['car insurance','maintenance', 'gas' 'bus', 'subway', 'train', 'ferry fare'],
    'food' : ['groceries', 'dining out', 'fast food', 'delivery', 'alcohol'],
    'utilities' : ['phone', 'cable', 'internet', 'water', 'gas', 'electricity', 'garbage', 'utility'], #oil, wood
    'medical' : ['health insurance','doctor', 'prescriptions', 'first aid supply'], #types of doctors, optometrist, dentist, etc
    'education' : ['supplies', 'fees', 'books'],
    'childcare' : ['daycare', 'camps', 'babysitter', 'child support'],
    'personal' : ['clothing', 'toiletries', 'grooming']
}

non_essentials = {
    'food' :['dining out', 'fast food', 'delivery', 'alcohol'],
    'gifts' : ['birthday', 'anniversary', 'holiday', 'other'],
    'giving': ['tithing', 'charitable', 'other'],
    'pets': ['food', 'supplies', 'vet', 'prescriptions'],
    'entertainment': ['streaming services','games', 'movies', 'hobbies', 'vacation', 'books'],
    'personal': ['salon', 'gym', 'dry cleaning']
    
}
#'domicile' : {'home': ['home insurance', 'mortgage', 'mortgage payment', 'home maitenance', 'home repair', 'property taxes', 'yard'],
#                'rental':['renters insurance', 'rent', 'rent payment', 'rental repair'],
#                 'furnishings':['furnishings', 'cleaning supply', 'kitchen ware','tools']},

# SUNBURST CHART

-------------------------------------------------------------------------------------------------------------------

In [ ]:
inside = ['savings', 'savings', 'debt', 'housing', 'transportation', 'food', 'utilities', 'medical', 'education', 'childcare', 'personal']

## Working
- go back to coolers.co and grab names of colors make a dictionary with the name as the key and color as value

In [ ]:
# red to blue/purple
rainbow_colors = ['#F94144', '#f3722c', '#F9844A', '#f8961e', '#F9C74F', '#90BE6D', '#43AA8B', '#4D908E', '#277DA1', '#577590']

In [ ]:
outside = ['essential' for i in range(len(inside))]

In [ ]:
# blues
# https://coolors.co/05668d-028090-00a896-02c39a-f0f3bd
# rainbow
# https://coolors.co/f94144-f3722c-f8961e-f9844a-f9c74f-90be6d-43aa8b-4d908e-577590-277da1

In [ ]:
import random
def constrain_sum(n, total):
    # Return an randomly chosen list of n positive intergers summing to 100/total
    dividers = sorted(random.sample(range(1, total), n-1))
    return [a - b for a,b in zip(dividers + [total], [0]+dividers)]
nums = constrain_sum(len(inside), 100)

In [ ]:
import random
import plotly.express as px
data = dict(
    character= inside,
    parent = outside,
    value= nums)

fig =px.sunburst(
    data,
    names='character',
    parents='parent',
    values='value',
    title = 'Expenses'
)
fig.show()

# TREEEMAP CHART

-------------------------------------------------------------------------------------------------------------------

In [ ]:
# Need new TreeMap code, this one is not so pretty

In [ ]:
essentials.keys()

In [ ]:
groups = list(essentials.keys())
priority = ['Essential' for i in range(len(groups))]
nums = constrain_sum(len(groups), 100)
percentages = [str(nums[i])+'%' for i in range(len(nums))]

In [ ]:
groups

In [ ]:
df = pd.DataFrame(
    {
        'priority': priority,
        'categories':groups,
        'values': nums,
        'percentages':percentages
                  })

In [ ]:
import squarify
import matplotlib.pyplot as plt
# plot it
labels = [df['categories'][i].capitalize()+(' \n'+ str(nums[i])+'%') for i in range(len(df['categories']))]

squarify.plot(sizes=df['values'], label=labels, alpha=.4, color= rainbow_colors[:len(groups)+1])
plt.axis('off')
plt.show()

# ALTERNATE TREEMAP

-------------------------------------------------------------------------------------------------------------------

- FIX Legend
- Change rainbow colors from a list to a dictionary and grab the correct values accordingly'
- ie if data 100-90 red, 90-80 orange, 80-70 yellow, 70-60 green, 60-50 blue, 50-40, purple and on until 10-0
- fix hover put something relevant

In [ ]:
groups_all = list(essentials.keys()) + list(non_essentials)
nums_all = constrain_sum(len(groups_all), 100)
priority_all = ['Essential' if x in essentials.keys() else 'Non Essential' for x in groups_all]
percentages_all = [str(nums_all[i])+'%' for i in range(len(nums_all))]
# Create a data frame with fake data
# df = pd.DataFrame({'nb_people':[8,3,4,2],
#                    'group':["Housing\n 40%",
#                             "Utilities\n 10%",
#                             "Food\n 10%",
#                             "Entertainment\n 10%"]})
df_all = pd.DataFrame(
    {
        'priority': priority_all,
        'categories':groups_all,
        'values': nums_all,
        'percentages':percentages_all
                  })

In [ ]:
colors_all = rainbow_colors[:len(groups_all)+1]
counter = len(colors_all)
while counter < len(df_all):
    colors_all.append(rainbow_colors[len(df_all)-counter])
    counter += 1
colors_all

In [ ]:
df_all

In [ ]:
sorted_df = df_all.sort_values(by=['values', 'categories']).reset_index(drop=True)
sorted_df

In [ ]:
import plotly.express as px
fig = px.treemap(df_all.sort_values(by=['values','categories']).reset_index(drop=True),
                 path=['priority', 'categories', 'percentages'], 
                 values='values', 
                 color='values')
fig.update_traces(marker_colors= colors_all, selector=dict(type='treemap')) 
#fig.update_traces(pathbar_visible=False, selector=dict(type='treemap'))
fig.show()

In [ ]:
month_categories.head()

In [ ]:
months = month_categories['month'].unique()
months = list(months)

In [ ]:
###
def get_percentage(month_num):
    perc = []
    mo_slice = month_categories[month_categories['month'] == month_num]
    sum_mo_1 = mo_slice['month_sum'].sum()
    for i in range(len(month_categories)):
        if month_categories['month'][i] == month_num:
            #print(round(month_categories['month_sum'][i], 2))
            #print(round(sum_mo_1)*100)
            amount = round(month_categories['month_sum'][i]/round(sum_mo_1)*100, 2)
            #print(amount)
            #break
            perc.append(amount)
    return perc

In [ ]:
percentage = []
final = []
for month in months:
    percentage.append(get_percentage(month))
final = [a for b in percentage for a in b]
month_categories['values'] = final

In [ ]:
january = month_categories.loc[month_categories['month'] == 1]

In [ ]:
def search_values(cat, cat_dict):
    for value in essentials.values():
        if cat in value:
            return True

In [ ]:
priority = []
for cat in january.category:
        if cat in essentials.keys() or search_values(cat, essentials):
            priority.append('Essential')
        else:
             priority.append('Non-Essential')

#            for entry in kind:
#               if cat in kind:
#                   priority.append('Essential')
#              else:      

In [ ]:
january['priority'] = priority

In [ ]:
january

In [ ]:
january['month_sum'] = january['month_sum'].apply(lambda x: x*-1)
january['percentages'] = january['values'].apply(lambda x : str(x)+'%')
january['month_sum'] = january['month_sum'].apply(lambda x : round(x, 2))
january

In [ ]:
type(january.month_sum[1])

In [ ]:
#colors_all

In [ ]:
from colour import Color
start = Color("#F94144")
colors = list(start.range_to(Color("#CAC35C"),20))
str(colors[1])


In [ ]:
color_list = []
for i in range(len(colors)):
    color_list.append(str(colors[i]))

In [ ]:
color

In [ ]:
from colr import color
print(color('test', fore='F94144', back='000'))

In [ ]:
for i in gradient:
    print(color('■', fore=i, back='000'), end='')


In [ ]:
for i in gradient:
    print(color(f'{i}', fore=i, back='000'), end='')


In [ ]:
#help('colour')

In [ ]:
for i in colors:
    print(color('■', fore=str(i), back='000'), end='')

In [ ]:
def make_treemap(df):
    import plotly.express as px
    df['month_sum_s'] = [f'${str(i)}' for i in df.month_sum]
    fig = px.treemap(df.sort_values(by=['values', 'category']).reset_index(drop=True), 
                     title= f"{df['long_month'][0]} Spending",
                     path=['priority','category','percentages'],
                     values='values', 
                     color= 'month_sum',
                     color_continuous_scale= gradient[::-1],
                     color_continuous_midpoint=np.average(df['month_sum'], weights=df['values']),
                     hover_name='month_sum')
    #fig.update_traces(marker_colors = colors_all, selector=dict(type='treemap'))
    fig.update_traces(hovertemplate='month_sum={month_sum}')
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    fig.show()
    
#px.colors.sequential.Viridis

In [ ]:
make_treemap(january)

In [ ]:
january.head()

In [ ]:
import plotly.express as px
fig = px.treemap(january.sort_values(by=['values', 'category']).reset_index(drop=True), 
                 path=['priority','percentages', 'category'], 
                 values='values', 
                 color='values',)
                 #title= f"{df['long_month'][0]} Spending",)
fig.update_traces(marker_colors= colors_all, selector=dict(type='treemap')) 
#fig.update_traces(pathbar_visible=False, selector=dict(type='treemap'))
#fig.update_traces(hovertemplate='category: '+f"{labels}" + '<br><extra></extra>')
fig.update_traces(marker_cmax=100, marker_cmid=50, marker_cmin=0, selector=dict(type='treemap'))
fig.show()

In [ ]:
import plotly.express as px
fig = px.treemap(df_all.sort_values(by=['values','categories']).reset_index(drop=True), 
                 path=['priority', 'categories', 'percentages'], 
                 values='values', 
                 color='values')
fig.update_traces(marker_colors= colors_all, selector=dict(type='treemap')) 
#fig.update_traces(pathbar_visible=False, selector=dict(type='treemap'))
fig.show()

In [ ]:
px.data.tips()

In [ ]:
import plotly.express as px
  
df = px.data.tips()
df.head()

In [ ]:
fig = px.treemap(df, path=['day', 'time', 'tip'], 
                 values='total_bill', 
                 color='total_bill')

fig.show()

In [ ]:
import plotly.express as px

for name in dir(px.data):
    if '__' not in name:
        print(name)

In [ ]:
print(px.data.tips())